In [1]:
import pandas as pd
from collections import defaultdict
import sys

In [2]:
finalfile="/Users/irffanalahi/Research/Research_code/gitignorefolder/MHB_per_read/moretest/readbaseddecon3/data/RealRun/collectmasterdf/sm_globaloutsubset/blwithneu_hypo_stat.txt_0.01_2_g100.txtWithSurrounding.txt_1cpgnc.txt_result/blwithneu_hypo_stat.txt_0.01_2_g100.txtWithSurrounding.txt_1cpgnc.txt_result_masterdf.txttemp_finalized/nocov_mincpg1/BULK-PBMC-1389.bg_howsm_single_mode_pp_masterdf.txt_rej-99999.0_mincpg1_rejmode_nov_final.txt"
sm="/Users/irffanalahi/Research/Research_code/gitignorefolder/MHB_per_read/moretest/readbaseddecon3/data/RealRun/sm/blwithneu_hypo_stat.txt_0.01_2_g100.txtWithSurrounding.txt_1cpgnc.txt"
#celllist=["'CD14'","'CD19'","'CD4'","'CD8'","'CD56'","'Neu'","'EPCAM'"] 

finaldf=pd.read_csv(finalfile,sep="\t")
finaldf['acceptedCpG'] = finaldf.acceptedCpG.apply(lambda x: x[1:-1].split(','))
finaldf['CheckedCpG'] = finaldf.CheckedCpG.apply(lambda x: x[1:-1].split(','))
finaldf['finalrejectedfor']=finaldf.finalrejectedfor.apply(lambda x: x[1:-1].split(','))

smdf=pd.read_csv(sm,sep="\t")
smdf['celltype']=smdf.celltype.apply(lambda x: x[1:-1].split(','))
finaldf.head()

,ReadName,allcelltype,ucelltype,assignedcelltype,CheckedCpG,mismatchCpG,notacceptedCpG,acceptedCpG,finalacceptedfor,finalrejectedfor,MateCIGARok
0,A00580:76:HFLG2DSXX:2:2252:8015:27680,[],[''],lowmapq,[],[''],[''],[''],notdetermined,[otdetermine],Y
1,A00118:116:HJ7VFDSXX:3:1502:16947:25895,[],[''],lowmapq,[],[''],[''],[''],notdetermined,[otdetermine],Y
2,A00580:76:HFLG2DSXX:4:1137:7853:36104,[],[''],lowmapq,[],[''],[''],[''],notdetermined,[otdetermine],Y
3,A00580:76:HFLG2DSXX:2:1623:3920:33458,[],[''],NotAssigned,[],[''],[''],[''],notdetermined,[otdetermine],Y
4,A00118:116:HJ7VFDSXX:3:2539:32226:16063,[],[''],lowmapq,[],[''],[''],[''],notdetermined,[otdetermine],Y


In [3]:
smdf.head()

,chrom,start,end,celltype,blockchr,blockstart,blockend
0,chr15,52052102,52052104,['CD14'],chr15,52051953,52052253
1,chr3,122060546,122060548,['CD14'],chr3,122060397,122060721
2,chr16,79097508,79097510,['CD14'],chr16,79097359,79097659
3,chr1,159431113,159431115,['CD14'],chr1,159430964,159431264
4,chr12,48336372,48336374,['CD14'],chr12,48336223,48336523


In [4]:
celllist=(smdf['celltype']).tolist()
celllist=[x for sublist in celllist for x in sublist]
celllist=list(set(celllist))
print(celllist)

["'CD14'", "'CD56'", "'CD8'", "'CD19'", "'CD4'", "'Neu'"]


In [5]:
############# needs checking############


def accfeatures(fdf,cells):
    outdict={}
    for cell in cells:
        print(cell)
        celldf=fdf[fdf['finalacceptedfor']==cell]
        correscpg=(celldf['acceptedCpG']).tolist()
        correscpg=list(map(frozenset,correscpg))
        
        correscpg=list(set(correscpg))
        outdict[cell]=correscpg
    #print(outdict)

    
def rejfeatures(fdf,sdf,cells):
    

In [6]:
accfeatures(finaldf,celllist)

'CD14'
'CD56'
'CD8'
'CD19'
'CD4'
'Neu'
{"'CD14'": [frozenset({"'chr19:32353348'", " 'chr19:32353243'"}), frozenset({"'chr13:50518758'"}), frozenset({" 'chr1:159431178'", "'chr1:159431221'"}), frozenset({" 'chr5:91301287'", "'chr5:91301285'"}), frozenset({"'chr2:174036585'"}), frozenset({" 'chr11:620961'", " 'chr11:620957'", " 'chr11:621016'", "'chr11:620986'"}), frozenset({"'chr1:165671635'", " 'chr1:165671653'"}), frozenset({" 'chr5:91301184'", " 'chr5:91301287'", "'chr5:91301141'"}), frozenset({" 'chr1:159431113'", " 'chr1:159431221'", "'chr1:159431103'", " 'chr1:159431178'"}), frozenset({"'chr11:17112734'", " 'chr11:17112663'"}), frozenset({" 'chr12:106246479'", "'chr12:106246451'"}), frozenset({"'chr5:156963355'"}), frozenset({" 'chr8:47431450'", "'chr8:47431497'"}), frozenset({"'chr3:17050654'"}), frozenset({"'chr4:138968955'", " 'chr4:138968940'"}), frozenset({" 'chr9:124203899'", " 'chr9:124203877'", "'chr9:124203927'", " 'chr9:124203889'"}), frozenset({"'chr1:113953247'"}), fro